In [ ]:
import json
from datetime import datetime, timedelta

# Create sample JSON data with 5 rows of timestamp and price
sample_data = [
    {
        "timestamp": "2025-06-24T15:56:10",
        "price": 1.37282
    },
    {
        "timestamp": "2025-06-24T15:56:15", 
        "price": 1.37285
    },
    {
        "timestamp": "2025-06-24T15:56:20",
        "price": 1.37281
    },
    {
        "timestamp": "2025-06-24T15:56:25",
        "price": 1.37278
    },
    {
        "timestamp": "2025-06-24T15:56:30",
        "price": 1.37276
    }
]

# Print as formatted JSON
print(json.dumps(sample_data, indent=2))

# Or access individual rows
print("\nIndividual rows:")
for i, row in enumerate(sample_data, 1):
    print(f"Row {i}: {row}")

In [ ]:
import time

# Loop through the data with 1-second delay and display each row
print("Starting data stream with 1-second intervals...\n")

for i, data_point in enumerate(sample_data, 1):
    print(f"Second {i}: {data_point}")
    
    # Wait 1 second before next data point (except for the last one)
    if i < len(sample_data):
        time.sleep(1)

print("\nData stream completed!")

In [ ]:
import matplotlib.pyplot as plt
import time
from datetime import datetime
import pandas as pd

# Set up interactive mode for live plotting
plt.ion()
fig, ax = plt.subplots(figsize=(12, 6))

# Initialize empty lists for plotting
timestamps = []
prices = []

# Start the live plot
print("Starting live price chart with 1-second intervals...\n")

for i, data_point in enumerate(sample_data, 1):
    # Convert timestamp string to datetime
    timestamp = pd.to_datetime(data_point['timestamp'])
    price = data_point['price']
    
    # Add to our data lists
    timestamps.append(timestamp)
    prices.append(price)
    
    # Clear and redraw the plot
    ax.clear()
    ax.plot(timestamps, prices, 'b-o', linewidth=2, markersize=6, label='Price')
    ax.set_title(f'Live Price Chart - Data Point {i}', fontsize=14)
    ax.set_xlabel('Time')
    ax.set_ylabel('Price')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Format x-axis to show time nicely
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Update the plot
    plt.draw()
    plt.pause(0.1)  # Small pause to allow plot to update
    
    # Print current data point
    print(f"Second {i}: {data_point}")
    print(f"Current price: {price}")
    
    # Wait 1 second before next data point (except for the last one)
    if i < len(sample_data):
        time.sleep(1)

print("\nLive chart completed!")
plt.ioff()  # Turn off interactive mode
plt.show()  # Show final plot

In [ ]:
import time
from pyecharts.charts import Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import pandas as pd
from IPython.display import display, HTML, clear_output

# Initialize data for live plotting
timestamps = []
prices = []

print("Starting live ECharts price chart with 1-second intervals...\n")

for i, data_point in enumerate(sample_data, 1):
    # Convert timestamp and extract data
    timestamp = pd.to_datetime(data_point['timestamp'])
    price = data_point['price']
    
    # Add to our data lists
    timestamps.append(timestamp.strftime('%H:%M:%S'))  # Format for display
    prices.append(price)
    
    # Create ECharts line chart
    chart = (
        Line(init_opts=opts.InitOpts(theme=ThemeType.DARK, width="800px", height="400px"))
        .add_xaxis(timestamps)
        .add_yaxis(
            "Price",
            prices,
            symbol="circle",
            symbol_size=8,
            linestyle_opts=opts.LineStyleOpts(width=3, color="#00ff88"),
            itemstyle_opts=opts.ItemStyleOpts(color="#00ff88"),
            markpoint_opts=opts.MarkPointOpts(
                data=[opts.MarkPointItem(type_="max"), opts.MarkPointItem(type_="min")]
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title=f"Live Price Chart - Data Point {i}",
                subtitle=f"Current: {price} | Time: {timestamp.strftime('%H:%M:%S')}",
                title_textstyle_opts=opts.TextStyleOpts(color="#ffffff")
            ),
            xaxis_opts=opts.AxisOpts(
                name="Time",
                axislabel_opts=opts.LabelOpts(rotate=45, color="#ffffff")
            ),
            yaxis_opts=opts.AxisOpts(
                name="Price",
                axislabel_opts=opts.LabelOpts(color="#ffffff")
            ),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(color="#ffffff")),
            datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=100)],
        )
    )
    
    # Clear previous output and display new chart
    clear_output(wait=True)
    
    # Print current data point
    print(f"Second {i}: {data_point}")
    print(f"Current price: {price}")
    
    # Display the chart
    display(HTML(chart.render_embed()))
    
    # Wait 1 second before next data point (except for the last one)
    if i < len(sample_data):
        time.sleep(1)

print("\nLive ECharts completed!")

In [ ]:
import time
import json
from datetime import datetime
import pandas as pd
from http.server import HTTPServer, BaseHTTPRequestHandler
import threading
import webbrowser
import os

# Global variables for data sharing
current_data = []
data_index = 0

class ChartHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            
            html = f"""
            <!DOCTYPE html>
            <html>
            <head>
                <title>Live Price Chart</title>
                <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
                <style>
                    body {{ 
                        font-family: Arial, sans-serif; 
                        background: #1a1a1a; 
                        color: white; 
                        margin: 0; 
                        padding: 20px;
                    }}
                    .container {{ 
                        max-width: 1200px; 
                        margin: 0 auto; 
                    }}
                    .stats {{ 
                        background: #2a2a2a; 
                        padding: 15px; 
                        border-radius: 8px; 
                        margin-bottom: 20px;
                        display: flex;
                        justify-content: space-between;
                    }}
                    .chart-container {{ 
                        background: #2a2a2a; 
                        padding: 20px; 
                        border-radius: 8px; 
                        height: 400px;
                    }}
                    h1 {{ text-align: center; color: #00ff88; }}
                    .stat-item {{ text-align: center; }}
                    .stat-value {{ font-size: 24px; font-weight: bold; color: #00ff88; }}
                    .stat-label {{ font-size: 14px; color: #aaa; }}
                </style>
            </head>
            <body>
                <div class="container">
                    <h1>🚀 Live Price Chart</h1>
                    
                    <div class="stats">
                        <div class="stat-item">
                            <div class="stat-value" id="currentPrice">-</div>
                            <div class="stat-label">Current Price</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-value" id="dataPoints">0</div>
                            <div class="stat-label">Data Points</div>
                        </div>
                        <div class="stat-item">
                            <div class="stat-value" id="lastUpdate">-</div>
                            <div class="stat-label">Last Update</div>
                        </div>
                    </div>
                    
                    <div class="chart-container">
                        <canvas id="priceChart"></canvas>
                    </div>
                </div>

                <script>
                    const ctx = document.getElementById('priceChart').getContext('2d');
                    const chart = new Chart(ctx, {{
                        type: 'line',
                        data: {{
                            labels: [],
                            datasets: [{{
                                label: 'Price',
                                data: [],
                                borderColor: '#00ff88',
                                backgroundColor: 'rgba(0, 255, 136, 0.1)',
                                borderWidth: 3,
                                pointBackgroundColor: '#00ff88',
                                pointBorderColor: '#ffffff',
                                pointBorderWidth: 2,
                                pointRadius: 6,
                                fill: true,
                                tension: 0.4
                            }}]
                        }},
                        options: {{
                            responsive: true,
                            maintainAspectRatio: false,
                            plugins: {{
                                legend: {{
                                    labels: {{
                                        color: 'white'
                                    }}
                                }}
                            }},
                            scales: {{
                                x: {{
                                    ticks: {{
                                        color: 'white'
                                    }},
                                    grid: {{
                                        color: 'rgba(255, 255, 255, 0.1)'
                                    }}
                                }},
                                y: {{
                                    ticks: {{
                                        color: 'white'
                                    }},
                                    grid: {{
                                        color: 'rgba(255, 255, 255, 0.1)'
                                    }}
                                }}
                            }},
                            animation: {{
                                duration: 750,
                                easing: 'easeInOutQuart'
                            }}
                        }}
                    }});

                    function updateChart() {{
                        fetch('/data')
                            .then(response => response.json())
                            .then(data => {{
                                if (data.length > 0) {{
                                    const labels = data.map(item => new Date(item.timestamp).toLocaleTimeString());
                                    const prices = data.map(item => item.price);
                                    
                                    chart.data.labels = labels;
                                    chart.data.datasets[0].data = prices;
                                    chart.update('none');
                                    
                                    // Update stats
                                    const latest = data[data.length - 1];
                                    document.getElementById('currentPrice').textContent = latest.price.toFixed(5);
                                    document.getElementById('dataPoints').textContent = data.length;
                                    document.getElementById('lastUpdate').textContent = new Date(latest.timestamp).toLocaleTimeString();
                                }}
                            }})
                            .catch(error => console.error('Error fetching data:', error));
                    }}

                    // Update every 500ms for smooth updates
                    setInterval(updateChart, 500);
                    
                    // Initial load
                    updateChart();
                </script>
            </body>
            </html>
            """
            self.wfile.write(html.encode())
            
        elif self.path == '/data':
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.send_header('Access-Control-Allow-Origin', '*')
            self.end_headers()
            self.wfile.write(json.dumps(current_data).encode())
        else:
            self.send_response(404)
            self.end_headers()

def start_server():
    """Start the HTTP server in a separate thread"""
    server = HTTPServer(('localhost', 8000), ChartHandler)
    print("🌐 Starting server at http://localhost:8000")
    server.serve_forever()

def stream_data():
    """Stream data with 1-second delay"""
    global current_data, data_index
    
    print("📊 Starting live data stream with 1-second intervals...\n")
    
    for i, data_point in enumerate(sample_data, 1):
        # Add new data point to current_data
        current_data.append(data_point)
        data_index = i
        
        # Print current data point
        print(f"Second {i}: {data_point}")
        print(f"📈 Chart updated at http://localhost:8000")
        print(f"Current price: {data_point['price']}")
        print("-" * 50)
        
        # Wait 1 second before next data point (except for the last one)
        if i < len(sample_data):
            time.sleep(1)
    
    print("\n✅ Live data stream completed!")
    print("🌐 Chart remains available at http://localhost:8000")

# Start the server in a background thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# Wait a moment for server to start
time.sleep(1)

# Open browser automatically
try:
    webbrowser.open('http://localhost:8000')
    print("🚀 Browser opened automatically!")
except:
    print("🌐 Please open http://localhost:8000 in your browser")

# Start streaming data
stream_data()

🚀 Browser opened automatically!
📊 Starting live data stream with 1-second intervals...

Second 1: {'timestamp': '2025-06-24T15:56:10', 'price': 1.37282}
📈 Chart updated at http://localhost:8000
Current price: 1.37282
--------------------------------------------------
Second 2: {'timestamp': '2025-06-24T15:56:15', 'price': 1.37285}
📈 Chart updated at http://localhost:8000
Current price: 1.37285
--------------------------------------------------
🌐 Starting server at http://localhost:8000
Second 3: {'timestamp': '2025-06-24T15:56:20', 'price': 1.37281}
📈 Chart updated at http://localhost:8000
Current price: 1.37281
--------------------------------------------------
Second 4: {'timestamp': '2025-06-24T15:56:25', 'price': 1.37278}
📈 Chart updated at http://localhost:8000
Current price: 1.37278
--------------------------------------------------
Second 5: {'timestamp': '2025-06-24T15:56:30', 'price': 1.37276}
📈 Chart updated at http://localhost:8000
Current price: 1.37276
--------------------

127.0.0.1 - - [25/Jun/2025 09:10:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:25] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:25] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Jun/2025 09:10:25] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:26] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:26] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:27] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:27] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:28] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:28] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:28] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:29] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 09:10:29] "GET /da